# Train Keypoints Classifier, i.e. Pose Classifier

In this notebook we are going to train our own pose classifier in PyTorch based on the dataset we built from the python script and notebook `00` and `01` respectively.

**Before you go any further** make sure you have already created and saved your csv file in `../data/my-data`.

Code adapted from this [repo](https://github.com/Alimustoofaa/YoloV8-Pose-Keypoint-Classification/tree/master).

First let's do some imports:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

If you are getting an error you may need to uncomment the next line to install sklearn:

In [ ]:
#!pip install scikit-learn

##### Hyperparameters

Now let's define our hyperparameters:

In [ ]:
device = 'cpu'
num_epochs = 200
num_classes = 3
test_size = 0.3
batch_size = 128
learn_rate = 0.001
data_path = '../data/my-data/poses_keypoints.csv'

##### Read Dataset

Here, we are reading the first 5 rows of our dataset.

In [ ]:
df = pd.read_csv(data_path)
df = df.drop('image_name', axis=1)
df.head()

##### Count and plot our data per class

In the following two cells, we are counting and plotting the number of data per class.

In [ ]:
df.label.value_counts()

In [ ]:
df.label.value_counts().plot(kind="bar")
plt.xticks(rotation=45)
plt.show()

##### Define the 1st column as our labels `y` and the following 34 columns as our keypoints input dataset `X`

In [ ]:
# Use the encoder label, to turn each label into an index number
encoder = LabelEncoder()
y_label = df['label']
y = encoder.fit_transform(y_label)
y

In [ ]:
# Get keypoint dataset 
X = df.iloc[:, 1:] # start from 11: if you want to skip the keipoints of the face
X

##### Train Test Split

Perform a train-test split with test_size=0.3 (defined in our hyperparameters), and a random but deterministic split and a strification.

Stratified sampling is a method of sampling that involves dividing a population into homogeneous subgroups known as strata, and then sampling from each stratum.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

print("Number of Training keypoints: ", len(X_train))
print("Number of Testing keypoints: ", len(X_test))

In [ ]:
# Optional step if you want to explore how the stratification creates homogenous subsets of data.

# from collections import Counter

# print(Counter(y))
# print(Counter(y_train))
# print(Counter(y_test))

In [ ]:
# A glimpse into the test data in a table format
X_test

##### MinMax scaling to scale each feature into a given range

For more information, look [here](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).

In [ ]:
# A glipse into the test data in the format of an array and after performing a minmax scaling
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_test

##### Data Loader

The data are currently numpy arrays and need to get transformed into torch tensors in order to get into the dataloaders.

In [ ]:
class DataKeypointClassification(Dataset):
    def __init__(self, X, y):
        self.x = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.int64))
        self.n_samples = X.shape[0]
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.n_samples

In [ ]:
train_dataset = DataKeypointClassification(X_train, y_train)
test_dataset = DataKeypointClassification(X_test, y_test)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size,  shuffle=False)

##### Define our simple feed forward network

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
      super(NeuralNet, self).__init__()
      self.fc1 = nn.Linear(X_train.shape[1], 256)
      self.fc2 = nn.Linear(256, num_classes)     
  
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=-1)
        return x

##### Setup core objects

Here we setup our core objects, the model, the loss function and the optimiser.

In [ ]:
model = NeuralNet()
model.to(device)

# Cross entropy loss for training classification
criterion = nn.CrossEntropyLoss()

# Adam optimiser
optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)

##### Training loop

Here is our training loop for our data.

In [ ]:
train_losses = []
best_loss = 100000
for epoch in range(num_epochs):
    train_loss = 0.0
    
    # Training loop
    for i, data in enumerate(train_loader, 0):
        # Get data
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Process data
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        
        # Update model weights
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    
    # Normalise cumulative losses to dataset size
    train_loss = train_loss / len(train_loader)
    
    # Added cumulative losses to lists for later display
    train_losses.append(train_loss)
    
    print(f'Epoch {epoch + 1}, train loss: {train_loss:.3f}')

##### Plot training loss

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Train loss")
plt.plot(train_losses,label="train")
plt.xlabel("epochs")
plt.ylabel("cumulative loss")
plt.legend()
plt.show()

##### Test our model

Here we use the model to predict the label on unseen data, our test data.

The predictions are the predicted classes (in the encoded format 0-1-2) for each item in the test dataset.

In [ ]:
test_features = torch.from_numpy(X_test.astype(np.float32))
test_labels = y_test
with torch.no_grad():
    outputs = model(test_features)
    _, predictions = torch.max(outputs, 1)
predictions

##### Confusion Matrix

A confusion matrix is a really good way to visualise the number of true positives, false negatives, false positives, and true negatives.

The header row corresponds to the predicted labels while the first column corresponds to the ground truth.

In [ ]:
cm = confusion_matrix(test_labels, predictions)
df_cm = pd.DataFrame(
    cm, 
    index = encoder.classes_,
    columns = encoder.classes_
)
df_cm

##### Visualising the confusion matrix with a seaborn heatmap

In [ ]:
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    plt.ylabel("Surface Ground Truth")
    plt.xlabel("Predicted Surface")
    plt.legend()
    
show_confusion_matrix(df_cm)

##### Save model


In [ ]:
PATH_SAVE = './pose_classifier.pt'
torch.save(model.state_dict(), PATH_SAVE)

##### Load Inference Model

In [ ]:
model_inference =  NeuralNet()
model_inference.load_state_dict(torch.load(PATH_SAVE, map_location=device))

In [ ]:
feature, label = test_dataset.__getitem__(12) # test out different item numbers

out = model_inference(feature)
_, predict = torch.max(out, -1)
print(f'\
    prediction label : {encoder.classes_[predict]} \n\
    ground thrut label : {encoder.classes_[label]}'
    )
print(encoder.classes_)

### Tasks:

**Task 1:** Run all the cells in this code to train your own pose classifier.

**Task 2:** Test out the training with different hyperparameters.

**Bonus Task 1:**

If you created more than one datasets from the previous notebook, test the same network architecture on your different training datasets and observe how the their size and the variations within their classes might affect the loss and the confusion matrices at the end of the training.

**Bonus Task 2:** 

You could perform an extra train-test split to create a validation subset and calculate the validation losses along with the train losses during the training, similar to what we did last week. Look into week 3 `train-image-classifier-from-scratch` for reference. 

To perform a second split, you will first need to split your input data into training and temp subsets and then, the temp subset into test and validation.